In [1]:
import sqlite3

from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from scipy.stats import fisk

In [9]:
class DataLoader:
    """
    Class responsible for loading data from an SQLite database.
    """

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.dataframes = {}

    def load_data(self) -> dict:
        """
        Load data from the SQLite database and store it in a dictionary.

        Returns:
            dict: A dictionary where keys are table names and values are DataFrames.
        """
        connection = sqlite3.connect(self.db_path)
        tables = [
            "drivers",
            "fcyphases",
            "laps",
            "qualifyings",
            "races",
            "retirements",
            "starterfields",
        ]
        self.dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", connection)
            for table in tables
        }
        connection.close()
        return self.dataframes



In [10]:
db_path = "F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()

In [8]:
laps_df = dataframes["laps"]
laps_race = laps_df[laps_df["race_id"] == 2]

# Trouver le dernier tour de chaque pilote
last_laps = laps_race.groupby("driver_id")["lapno"].max().reset_index()

# Récupérer le temps de course associé au dernier tour de chaque pilote
final_times = pd.merge(laps_race, last_laps, on=["driver_id", "lapno"], how="inner")[["driver_id", "racetime"]]
final_times = final_times.rename(columns={"racetime": "cumulative_time_actual"})


In [9]:
final_times

,driver_id,cumulative_time_actual
0,16,0.000
1,21,808.607
2,18,1029.632
3,6,2089.905
4,13,3565.971
5,20,3897.937
6,2,5522.546
7,19,6037.511
8,17,6037.641
9,4,6028.365


In [26]:
import pandas as pd
import numpy as np

def generate_pit_stop_strategy(season: int, location: str, dataframes: dict):
    """
    Generates pit stop strategies for each driver in a given season and location.
    
    Args:
        season (int): The racing season.
        location (str): Grand Prix location.
        dataframes (dict): Dictionary containing race-related data.
    
    Returns:
        dict: Pit stop strategies for each driver.
    """
    # Load required data from the dataframes
    df_pit_stops = dataframes.get("laps", pd.DataFrame())
    df_races = dataframes.get("races", pd.DataFrame())
    df_drivers = dataframes.get("drivers", pd.DataFrame())
    
    # Find the race ID for the given season and location
    race_row = df_races[(df_races["season"] == season) & (df_races["location"] == location)]
    if race_row.empty:
        raise ValueError(f"No race found for location '{location}' in season {season}.")
    
    race_id = race_row.iloc[0]["id"]
    
    # Filter pit stop data for the selected race
    pit_stop_data = df_pit_stops[df_pit_stops["race_id"] == race_id]
    
    # Extract unique drivers from the filtered data
    drivers = pit_stop_data["driver_id"].unique()
    
    pit_stop_strategies = {}
    
    for driver_id in drivers:
        driver_pit_stops = pit_stop_data[pit_stop_data["driver_id"] == driver_id]
        driver_name = df_drivers[df_drivers["id"] == driver_id]["name"].values[0]
        
        # Find the starting tire compound and tire age (lap 0)
        starting_tire = driver_pit_stops[driver_pit_stops["lapno"] == 0]
        if not starting_tire.empty:
            starting_compound = starting_tire.iloc[0]["compound"]
            starting_tire_age = starting_tire.iloc[0]["tireage"]
        else:
            starting_compound = "Unknown"
            starting_tire_age = "Unknown"
        
        strategy = {"starting_compound": starting_compound, "starting_tire_age": starting_tire_age}
        
        # Filter only actual pit stops (where pitstopduration is not NaN or zero)
        valid_pit_stops = driver_pit_stops[driver_pit_stops["pitstopduration"].notna() & (driver_pit_stops["pitstopduration"] > 0)]
        
        for idx, (lap, duration, compound, tireage) in enumerate(
            zip(valid_pit_stops["lapno"], valid_pit_stops["pitstopduration"], valid_pit_stops["compound"], valid_pit_stops["tireage"])
        ):
            pitstop_interval = [lap,lap]
            strategy[idx+1] = {
                "compound": compound,
                "pitstop_interval": pitstop_interval,
                "pit_stop_lap": lap,
                "tire_age": tireage
            }
        
        pit_stop_strategies[driver_name] = strategy
    
    return pit_stop_strategies

generate_pit_stop_strategy(season=2016, location="Austin",dataframes=dataframes)


{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tire_age': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tire_age': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tire_age': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tire_age': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tire_age': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tire_age': 11},
  2: {'comp

In [ ]:
{'Lewis Hamilton': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 20}},
 'Nico Rosberg': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 12},
  2: {'compound': 'A2',
   'pitstop_interval': [31, 31],
   'pit_stop_lap': 31,
   'tireage': 21}},
 'Daniel Ricciardo': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [25, 25],
   'pit_stop_lap': 25,
   'tireage': 17}},
 'Max Verstappen': {'starting_compound': 'A3',
  'starting_tire_age': 2,
  1: {'compound': 'A3',
   'pitstop_interval': [9, 9],
   'pit_stop_lap': 9,
   'tireage': 11},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 17}},
 'Kimi Raikkonen': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [8, 8],
   'pit_stop_lap': 8,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [24, 24],
   'pit_stop_lap': 24,
   'tireage': 16},
  3: {'compound': 'A4',
   'pitstop_interval': [38, 38],
   'pit_stop_lap': 38,
   'tireage': 14}},
 'Sebastian Vettel': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [14, 14],
   'pit_stop_lap': 14,
   'tireage': 16},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 15},
  3: {'compound': 'A2',
   'pitstop_interval': [53, 53],
   'pit_stop_lap': 53,
   'tireage': 24}},
 'Nico Hulkenberg': {'starting_compound': 'A4', 'starting_tire_age': 2},
 'Valtteri Bottas': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [1, 1],
   'pit_stop_lap': 1,
   'tireage': 3},
  2: {'compound': 'A3',
   'pitstop_interval': [20, 20],
   'pit_stop_lap': 20,
   'tireage': 19}},
 'Felipe Massa': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 18},
  3: {'compound': 'A2',
   'pitstop_interval': [54, 54],
   'pit_stop_lap': 54,
   'tireage': 25}},
 'Carlos Sainz Jnr': {'starting_compound': 'A4',
  'starting_tire_age': 2,
  1: {'compound': 'A4',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Sergio Perez': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Fernando Alonso': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [11, 11],
   'pit_stop_lap': 11,
   'tireage': 11},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 19}},
 'Daniil Kvyat': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [21, 21],
   'pit_stop_lap': 21,
   'tireage': 21}},
 'Esteban Gutierrez': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13}},
 'Jolyon Palmer': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [15, 15],
   'pit_stop_lap': 15,
   'tireage': 15},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 11}},
 'Marcus Ericsson': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17}},
 'Romain Grosjean': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 17}},
 'Kevin Magnussen': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A3',
   'pitstop_interval': [27, 27],
   'pit_stop_lap': 27,
   'tireage': 14},
  3: {'compound': 'A2',
   'pitstop_interval': [43, 43],
   'pit_stop_lap': 43,
   'tireage': 16}},
 'Jenson Button': {'starting_compound': 'A4',
  'starting_tire_age': 0,
  1: {'compound': 'A4',
   'pitstop_interval': [10, 10],
   'pit_stop_lap': 10,
   'tireage': 10},
  2: {'compound': 'A2',
   'pitstop_interval': [28, 28],
   'pit_stop_lap': 28,
   'tireage': 18}},
 'Pascal Wehrlein': {'starting_compound': 'A3',
  'starting_tire_age': 0,
  1: {'compound': 'A3',
   'pitstop_interval': [13, 13],
   'pit_stop_lap': 13,
   'tireage': 13},
  2: {'compound': 'A2',
   'pitstop_interval': [30, 30],
   'pit_stop_lap': 30,
   'tireage': 17}},
 'Felipe Nasr': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [29, 29],
   'pit_stop_lap': 29,
   'tireage': 29}},
 'Esteban Ocon': {'starting_compound': 'A2',
  'starting_tire_age': 0,
  1: {'compound': 'A2',
   'pitstop_interval': [17, 17],
   'pit_stop_lap': 17,
   'tireage': 17},
  2: {'compound': 'A3',
   'pitstop_interval': [26, 26],
   'pit_stop_lap': 26,
   'tireage': 9},
  3: {'compound': 'A3',
   'pitstop_interval': [44, 44],
   'pit_stop_lap': 44,
   'tireage': 18}}}

In [8]:
from monte_carlo_simulator import MonteCarloSimulator

db_path = "F1_timingdata_2014_2019.sqlite"
season = 2016
gp_location = "Austin"
num_simulations = 2000

driver_strategies = {
    'Lewis Hamilton': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [31, 31],
                            'pit_stop_lap': 31,
                            'tire_age': 20}},
    'Nico Rosberg': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 12},
                        2: {'compound': 'A2',
                            'pitstop_interval': [31, 31],
                            'pit_stop_lap': 31,
                            'tire_age': 21}},
    'Daniel Ricciardo': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [8, 8],
                                'pit_stop_lap': 8,
                                'tire_age': 10},
                            2: {'compound': 'A3',
                                'pitstop_interval': [25, 25],
                                'pit_stop_lap': 25,
                                'tire_age': 17}},
    'Max Verstappen': {'starting_compound': 'A3',
                        'starting_tire_age': 2,
                        1: {'compound': 'A3',
                            'pitstop_interval': [9, 9],
                            'pit_stop_lap': 9,
                            'tire_age': 11},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 17}},
    'Kimi Raikkonen': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [8, 8],
                            'pit_stop_lap': 8,
                            'tire_age': 10},
                        2: {'compound': 'A3',
                            'pitstop_interval': [24, 24],
                            'pit_stop_lap': 24,
                            'tire_age': 16},
                        3: {'compound': 'A4',
                            'pitstop_interval': [38, 38],
                            'pit_stop_lap': 38,
                            'tire_age': 14}},
    'Sebastian Vettel': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [14, 14],
                                'pit_stop_lap': 14,
                                'tire_age': 16},
                            2: {'compound': 'A3',
                                'pitstop_interval': [29, 29],
                                'pit_stop_lap': 29,
                                'tire_age': 15},
                            3: {'compound': 'A2',
                                'pitstop_interval': [53, 53],
                                'pit_stop_lap': 53,
                                'tire_age': 24}},
    'Nico Hulkenberg': {'starting_compound': 'A4', 'starting_tire_age': 2},
    'Valtteri Bottas': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [1, 1],
                            'pit_stop_lap': 1,
                            'tire_age': 3},
                        2: {'compound': 'A3',
                            'pitstop_interval': [20, 20],
                            'pit_stop_lap': 20,
                            'tire_age': 19}},
    'Felipe Massa': {'starting_compound': 'A4',
                        'starting_tire_age': 2,
                        1: {'compound': 'A4',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [29, 29],
                            'pit_stop_lap': 29,
                            'tire_age': 18},
                        3: {'compound': 'A2',
                            'pitstop_interval': [54, 54],
                            'pit_stop_lap': 54,
                            'tire_age': 25}},
    'Carlos Sainz Jnr': {'starting_compound': 'A4',
                            'starting_tire_age': 2,
                            1: {'compound': 'A4',
                                'pitstop_interval': [11, 11],
                                'pit_stop_lap': 11,
                                'tire_age': 13},
                            2: {'compound': 'A3',
                                'pitstop_interval': [30, 30],
                                'pit_stop_lap': 30,
                                'tire_age': 19}},
    'Sergio Perez': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A2',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 17}},
    'Fernando Alonso': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [11, 11],
                            'pit_stop_lap': 11,
                            'tire_age': 11},
                        2: {'compound': 'A2',
                            'pitstop_interval': [30, 30],
                            'pit_stop_lap': 30,
                            'tire_age': 19}},
    'Daniil Kvyat': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [21, 21],
                            'pit_stop_lap': 21,
                            'tire_age': 21}},
    'Esteban Gutierrez': {'starting_compound': 'A3',
                            'starting_tire_age': 0,
                            1: {'compound': 'A3',
                                'pitstop_interval': [13, 13],
                                'pit_stop_lap': 13,
                                'tire_age': 13}},
    'Jolyon Palmer': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [15, 15],
                            'pit_stop_lap': 15,
                            'tire_age': 15},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 11}},
    'Marcus Ericsson': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [17, 17],
                            'pit_stop_lap': 17,
                            'tire_age': 17}},
    'Romain Grosjean': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A3',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 17}},
    'Kevin Magnussen': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [13, 13],
                            'pit_stop_lap': 13,
                            'tire_age': 13},
                        2: {'compound': 'A3',
                            'pitstop_interval': [27, 27],
                            'pit_stop_lap': 27,
                            'tire_age': 14},
                        3: {'compound': 'A2',
                            'pitstop_interval': [43, 43],
                            'pit_stop_lap': 43,
                            'tire_age': 16}},
    'Jenson Button': {'starting_compound': 'A4',
                        'starting_tire_age': 0,
                        1: {'compound': 'A4',
                            'pitstop_interval': [10, 10],
                            'pit_stop_lap': 10,
                            'tire_age': 10},
                        2: {'compound': 'A2',
                            'pitstop_interval': [28, 28],
                            'pit_stop_lap': 28,
                            'tire_age': 18}},
    'Pascal Wehrlein': {'starting_compound': 'A3',
                        'starting_tire_age': 0,
                        1: {'compound': 'A3',
                            'pitstop_interval': [13, 13],
                            'pit_stop_lap': 13,
                            'tire_age': 13},
                        2: {'compound': 'A2',
                            'pitstop_interval': [30, 30],
                            'pit_stop_lap': 30,
                            'tire_age': 17}},
    'Felipe Nasr': {'starting_compound': 'A2',
                    'starting_tire_age': 0,
                    1: {'compound': 'A2',
                        'pitstop_interval': [29, 29],
                        'pit_stop_lap': 29,
                        'tire_age': 29}},
    'Esteban Ocon': {'starting_compound': 'A2',
                        'starting_tire_age': 0,
                        1: {'compound': 'A2',
                            'pitstop_interval': [17, 17],
                            'pit_stop_lap': 17,
                            'tire_age': 17},
                        2: {'compound': 'A3',
                            'pitstop_interval': [26, 26],
                            'pit_stop_lap': 26,
                            'tire_age': 9},
                        3: {'compound': 'A3',
                            'pitstop_interval': [44, 44],
                            'pit_stop_lap': 44,
                            'tire_age': 18}}
    }



In [9]:
# RUN Simulation 
simulator = MonteCarloSimulator(season, gp_location, db_path, driver_strategies, num_simulations)
simulator.run_simulation()
simulator.summarize()  # Compute comparison dataframe

simulator.comparison_df


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 1/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 2/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


...

Simulation 858/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


Simulation 859/2000 completed.


c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project\New\run.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.laps_summary = pd.concat([self.laps_summary, pd.DataFrame([new_row])], ignore_index=True)


KeyboardInterrupt: 

In [16]:
import pandas as pd

In [20]:
simulator.comparison_df
simulator.summarize()


=== Wilcoxon Test for Lap Times ===

📊 Résultats du test de Wilcoxon:
Statistique de test : 98.0
P-value : 0.37053
✅ Non-rejet de H0 : Aucune différence significative entre les temps simulés et réels.

=== Spearman Rank Correlation for Positions ===

📊 Résultats du test de corrélation de Spearman:
Coefficient de corrélation (rs) : 0.0751
P-value : 0.73977
✅ Non-rejet de H0 : Aucune corrélation significative entre les positions simulées et réelles.


In [28]:
comparison_dfTest=simulator.comparison_df
comparison_dfTest['rank_sim'] = comparison_dfTest['final_position_sim'].rank(method='dense', ascending=True)


,driver_id,final_position_sim,cumulative_time_sim,final_position_actual,cumulative_time_actual,rank_sim
0,1,6.644936,5415.843655,1,5892.618,2.0
1,2,7.981374,5529.615413,3,5912.310,6.0
2,3,8.997672,5392.873276,2,5897.138,8.0
3,4,10.493597,5407.930852,12,5943.131,10.0
4,5,14.491269,5433.107286,5,5986.571,17.0
5,7,11.133877,5251.110188,22,143.182,11.0
6,8,12.641444,5408.812003,11,5947.442,14.0
7,9,6.696158,5277.910317,7,5899.087,3.0
8,10,17.104773,5660.709342,9,5915.617,19.0
9,11,7.548312,5333.543555,19,4052.749,5.0


In [33]:
len(simulator.final_outcomes)/22

859.0

In [31]:
comparison_dfTest=comparison_dfTest.sort_values(by=["final_position_actual"])
comparison_dfTest

,driver_id,final_position_sim,cumulative_time_sim,final_position_actual,cumulative_time_actual,rank_sim
0,1,6.644936,5415.843655,1,5892.618,2.0
2,3,8.997672,5392.873276,2,5897.138,8.0
1,2,7.981374,5529.615413,3,5912.310,6.0
10,12,11.392317,5535.964641,4,5935.752,12.0
4,5,14.491269,5433.107286,5,5986.571,17.0
16,25,14.335274,5480.794361,6,5988.742,16.0
7,9,6.696158,5277.910317,7,5899.087,3.0
12,16,17.373690,5577.339212,8,5913.660,20.0
8,10,17.104773,5660.709342,9,5915.617,19.0
15,22,15.185099,5480.771201,10,5923.518,18.0
